In [1]:
import sys; sys.path.append("../../../../automl/")

In [2]:
from pathlib import Path
import yaml
import joblib

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from src.automl.model.lama import TabularLamaUtilized
from src.automl.loggers import configure_root_logger
from src.automl.constants import create_ml_data_dir
from src.automl.model.metrics import RocAuc

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
[2024-11-08 11:07:08]
/home/peter/venvs/base_venv/lib/python3.10/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/

In [3]:
create_ml_data_dir()
configure_root_logger()

## Constants

In [4]:
RANDOM_SEED = 77
DATA_PATH = Path("../../../../data/")
CONFIG_PATH = Path("../../../../configs/config.yaml")
N_JOBS = 16

In [5]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

In [6]:
cfg["stack_features"] = ["lamau_814_full_dataset", "lamann_autoint_8053_full_dataset"]

## Data

In [7]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc_oof.parquet")
#df_train, df_test = df_train.sort_values(by="id").iloc[:300_000], df_train.sort_values(by="id").iloc[300_000:]

In [8]:
df_train["target"].value_counts(normalize=True)

0    0.940982
1    0.059018
Name: target, dtype: float64

In [9]:
# undersample the 0 class
#df_train = pd.concat([df_train.loc[df_train.target == 1], df_train.loc[df_train.target == 0].sample(200_000, random_state=RANDOM_SEED)], ignore_index=True)


In [10]:
cat_columns = df_train.drop(columns=["target", "id"]).select_dtypes(int).columns.values.tolist()

In [11]:
X_train, y_train = df_train[cfg["selected_features"] + cfg["stack_features"] + cat_columns], df_train["target"]
#X_test, y_test = df_test[cfg["selected_features"] + cat_columns], df_test["target"]

In [12]:
display(y_train.value_counts(normalize=True))
#display(y_test.value_counts(normalize=True))

0    0.940982
1    0.059018
Name: target, dtype: float64

In [13]:
#categorical_features = ohe_cols# + oe_cols

## Model

In [14]:
metric = RocAuc()

In [15]:
# save_std_out = sys.stdout
# sys.stdout = open("stdout_logs", "w")

In [16]:
model = TabularLamaUtilized(n_jobs=16, task="classification")
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-08 11:07:39,035] - [   START    ] - Fitting TabularLamaUtilized
[11:07:39] Start automl utilizator with listed constraints:
[11:07:39] - time: 7200.00 seconds
[11:07:39] - CPU: 16 cores
[11:07:39] - memory: 16 GB

[11:07:39] If one preset completes earlier, next preset configuration will be started

[11:07:39] ==================================================
[11:07:39] Start 0 automl preset configuration:
[11:07:39] conf_0_sel_type_0.yml, random state: {'reader_params': {'random_state': 42}, 'nn_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
[11:07:39] Stdout logging level is INFO.
[11:07:39] Task: binary

[11:07:39] Start automl preset with listed constraints:
[11:07:39] - time: 7200.00 seconds
[11:07:39] - CPU: 16 cores
[11:07:39] - memory: 16 GB

[11:07:39] Train data shape: (413194, 65)

[11:07:50] Layer 1 train process start. Time left 7188.33 secs
[11:08:03] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[11:08:48] Fitting Lvl_0_Pipe

[11:35:41] Layer 1 train process start. Time left 5518.17 secs
[11:35:54] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[11:36:37] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.8068739902274549
[11:36:37] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[11:36:37] Time left 5461.96 secs

[11:36:43] Selector_LightGBM fitting and predicting completed
[11:36:55] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[11:37:26] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.8059020253143211
[11:37:26] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[11:37:26] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ... Time budget is 300.00 secs
[11:42:33] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[11:42:33] Start fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ...
[11:43:10] Fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM finished. score = 0.8139220517298654
[11:43:10] Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM fitting and p

[12:10:14] Start fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ...
[12:10:43] Fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM finished. score = 0.8111058075522293
[12:10:43] Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM fitting and predicting completed
[12:10:43] Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...
[12:11:06] Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = 0.8122375014101819
[12:11:06] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
[12:11:06] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 300.00 secs
[12:16:09] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[12:16:09] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...
[12:16:43] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.8123799805016096
[12:16:43] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[12:16:43] Time left 3056.14 secs

[12:16:43] Layer 1 training completed.

[12:16:43] Blending: optimization starts with e

[12:26:48] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...
[12:27:06] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.8105502893202271
[12:27:06] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[12:27:06] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...
[12:27:30] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.8123630418868147
[12:27:30] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[12:27:30] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...
[12:27:51] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.8117281667776433
[12:27:51] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[12:27:51] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...
[12:28:16] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.8123335226303843
[12:28:16] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[12:28:16] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...
[12:28:41] Fi

[12:45:19] Task: binary

[12:45:19] Start automl preset with listed constraints:
[12:45:19] - time: 1339.45 seconds
[12:45:19] - CPU: 16 cores
[12:45:19] - memory: 16 GB

[12:45:19] Train data shape: (413194, 65)

[12:45:30] Layer 1 train process start. Time left 1328.36 secs
[12:45:43] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[12:46:28] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.8067974022241808
[12:46:28] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[12:46:28] Time left 1270.79 secs

[12:46:40] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[12:47:11] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.8029268520676753
[12:47:11] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[12:47:11] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ... Time budget is 197.75 secs
[12:50:32] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[12:50:32] Start fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM 

In [17]:
print(metric(y_train, oof))

0.8140217779655352


In [19]:
model.model.get_feature_scores()

,Feature,Importance
0,lamau_814_full_dataset,155900.645331
1,feature_83,1862.207562
2,feature_14,1780.640471
3,feature_24,1777.973118
4,feature_148,1773.221087
...,...,...
59,feature_122,20.285800
60,feature_179,19.531767
61,feature_156,17.388200
62,feature_109,16.693633


In [20]:
MODEL_NAME = "lamau_stack_8140_full_dataset"
MODEL_DIR = Path(f"../../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [21]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
#joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_oof.parquet")
test["target"] = model.predict(test[cfg["selected_features"]+ cfg["stack_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

In [24]:
test = pd.read_parquet(DATA_PATH / "test_preproc_oof.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cfg["stack_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(f'{MODEL_NAME}.csv', index=False)

In [26]:
model.model.get_feature_scores()[:50]

,Feature,Importance
0,lamau_814_full_dataset,78048.337071
1,xgb_81325_full_dataset,29297.001751
2,lama_81298_full_dataset,24098.444188
3,feature_162,1398.695796
4,feature_24,1321.674997
5,feature_18,1298.269506
6,feature_26,1290.120701
7,feature_145,1287.683197
8,feature_78,1279.916003
9,feature_36,1248.758602


In [24]:
imp = pd.DataFrame().assign(names=model.models[0].feature_names_, imp=model.models[0].feature_importances_)

In [41]:
imp.sort_values(by="imp", ascending=False).reset_index(drop=True).query("names == 'feature_185'")

,names,imp
61,feature_185,0.0


In [29]:
cat_columns

['feature_7',
 'feature_31',
 'feature_60',
 'feature_61',
 'feature_71',
 'feature_109',
 'feature_122',
 'feature_156',
 'feature_163',
 'feature_167',
 'feature_179',
 'feature_185']

## With Time series cross val

In [13]:
metric = RocAuc()

In [14]:
df_train = df_train.sort_values(by="id").reset_index(drop=True)
X_train, y_train = df_train[cfg["selected_features"] + cat_columns], df_train["target"]

In [15]:
model = LightGBMClassification(n_jobs=16, time_series=True)
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-07 08:27:33,388] - [   START    ] - Tuning LightGBMClassification
[2024-11-07 08:27:40,118] - [   OPTUNA   ] - Trial 0. New best score 0.7903405446081995 with parameters {'max_depth': 6, 'num_leaves': 488, 'min_data_in_leaf': 188, 'bagging_fraction': 0.7993292420985183, 'bagging_freq': 0, 'feature_fraction': 0.49359671220172163, 'lambda_l1': 0.5808361216819946, 'lambda_l2': 8.661761457749352, 'min_gain_to_split': 12.022300234864176, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:03,043] - [   OPTUNA   ] - Trial 2. New best score 0.7913515589848906 with parameters {'max_depth': 5, 'num_leaves': 194, 'min_data_in_leaf': 117, 'bagging_fraction': 0.8925879806965068, 'bagging_freq': 0, 'feature_fraction': 0.708540663048167, 'lambda_l1': 5.924145688620425, 'lambda_l2': 0.46450412719997725, 'min_gain_to_split': 12.150897038028766, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:12,225] - [   OPTUNA   ] - Trial 3. New best score 0.8048686053278628 with param

In [27]:
none_oofs_idx = oof[np.any(np.isnan(oof), axis=1)].shape[0]

In [32]:
metric(y_train[none_oofs_idx:], oof[none_oofs_idx:])

0.8095227594190041

In [34]:
MODEL_NAME = "lgb_8095_full_dataset_time_series"
MODEL_DIR = Path(f"../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [35]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[none_oofs_idx:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

## TEST 
**81.22112399468679**

## Inference

In [27]:
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv('lgb_813.csv', index=False)

In [25]:
pred_1 = pd.read_csv("lama_utilized.csv")
pred_2 = pd.read_csv("lgmb_oe_ohe_cols_0805.csv")
pred_3 = pd.read_csv("catboost_ts.csv")

In [26]:
pred_1["target"] = 0.6 * pred_1["target"] + 0.2 * pred_2["target"] + 0.2 * pred_3["target"]

In [29]:
pred_1.to_csv("blend.csv", index=False)

In [166]:
MODEL_DIR.open?

Signature:
MODEL_DIR.open(
    mode='r',
    buffering=-1,
    encoding=None,
    errors=None,
    newline=None,
)
Docstring:
Open the file pointed by this path and return a file object, as
the built-in open() function does.
File:      /usr/lib/python3.10/pathlib.py
Type:      method